In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load data
df_gdp_mainlandchina = pd.read_csv('raw_data/data-gdp-mainlandchina-2019.csv')  
df_gdp_usa = pd.read_csv('raw_data/data-gdp-usa-2019.csv')  
df_pop_china = pd.read_csv('raw_data/data-pop-china-2017.csv')  
df_pop_usa = pd.read_csv('raw_data/data-pop-usa-2019.csv')  
df_WEO = pd.read_csv('raw_data/WEO_Data.csv')  
df_region_china = pd.read_csv('raw_data/data-economyregion-china.csv')  
df_continent = pd.read_csv('raw_data/country-and-continent-codes-list.csv') 
df_pop_europe = pd.read_csv('raw_data/data-pop-europe-2019.csv') 

# Get data of China

In [3]:
# Merge dataframe for China
df_china = pd.DataFrame.merge(df_gdp_mainlandchina,df_pop_china,how='right',left_on='Provinces',right_on='Administrative Division')
df_china = pd.DataFrame.merge(df_china,df_region_china,how='left',left_on='Administrative Division',right_on='Provinces')

# Don't show Unnamed columns
df_china = df_china.loc[:, ~df_china.columns.str.contains('^Unnamed')] 

# Drop column
df_china = df_china.drop(['Provinces_x'], axis=1)
df_china = df_china.drop(['Provinces_y'], axis=1)

# Add column
df_china['Country'] = 'China'
df_china['Year'] = '2019'

# Concat 'CN' to all value in[Region]
df_china['Region'] = df_china['Region'].map(str) + '-CN'

df_china.tail()

,Nominal GDP (Billion),Administrative Division,Population,Region,Economic Zone #,Economic Zone,经济区,Characteristic,Country,Year
29,43.0,Qinghai,5980000,West-CN,10,Qinghai-Tibet Plateau,青藏高原经济区,"Natural gas, salt lake resources, non-ferrous ...",China,2019
30,25.0,Tibet,3370000,West-CN,10,Qinghai-Tibet Plateau,青藏高原经济区,"Natural gas, salt lake resources, non-ferrous ...",China,2019
31,NaN,Hong Kong,7335384,Hong Kong-CN,11,Hong Kong,香港特别行政区,"Trade and logistics industry (21.2%), financia...",China,2019
32,NaN,Macau,644900,Macau-CN,12,Macau,澳门港特别行政区,Tourism and apparel industry (41%),China,2019
33,NaN,Taiwan,23562318,Taiwan-CN,13,Taiwan,台澎金马个别关税领域,Agricultural products and OEM,China,2019


In [4]:
# Get GDP value for Hong Kong, Macau and Taiwan
df_hongkong = df_WEO[df_WEO['Country'].str.contains('hong kong', case=False, na=False)]
hongkong_gdp = df_hongkong.loc[df_hongkong['Subject Descriptor'] == 'Gross domestic product, current prices', '2019'].values[0]

df_macau = df_WEO[df_WEO['Country'].str.contains('maca', case=False, na=False)]
macau_gdp = df_macau.loc[df_macau['Subject Descriptor'] == 'Gross domestic product, current prices', '2019'].values[0]

df_taiwan = df_WEO[df_WEO['Country'].str.contains('taiwan', case=False, na=False)]
taiwan_gdp = df_taiwan.loc[df_taiwan['Subject Descriptor'] == 'Gross domestic product, current prices', '2019'].values[0]

print('Hong Kong GDP: ',hongkong_gdp)
print('Macau GDP: ',macau_gdp)
print('Taiwan GDP: ',taiwan_gdp)

Hong Kong GDP:  372.99
Macau GDP:  55.14
Taiwan GDP:  586.1


In [5]:
# Assign value
## to Hong Kong
hongkong_index = df_china.loc[df_china['Administrative Division'] == 'Hong Kong'].index[0]
df_china.at[hongkong_index, 'Nominal GDP (Billion)'] = hongkong_gdp

## to Macau
macau_index = df_china.loc[df_china['Administrative Division'] == 'Macau'].index[0]
df_china.at[macau_index, 'Nominal GDP (Billion)'] = macau_gdp

## to Taiwan 
taiwan_index = df_china.loc[df_china['Administrative Division'] == 'Taiwan'].index[0]
df_china.at[taiwan_index, 'Nominal GDP (Billion)'] = taiwan_gdp

# Calculate GDP per Capita
df_china['GDP per Capita'] = (df_china['Nominal GDP (Billion)'] / df_china['Population'])*1000000000

# Re-order Columns
df_china = df_china[['Year','Country','Region','Economic Zone #','Economic Zone','Administrative Division','Nominal GDP (Billion)','Population','GDP per Capita','Characteristic']]

# Set 0 decimal places
df_china['GDP per Capita'] = df_china['GDP per Capita'].round(0)

df_china

,Year,Country,Region,Economic Zone #,Economic Zone,Administrative Division,Nominal GDP (Billion),Population,GDP per Capita,Characteristic
0,2019,China,East-CN,4,Southeast Coast,Guangdong,1561.00,111690000,13976.0,"Export-oriented, digesting foreign advanced te..."
1,2019,China,East-CN,3,East Coast,Jiangsu,1444.00,80290000,17985.0,Multifunctional manufacturing: Light industria...
2,2019,China,East-CN,2,North Coast,Shandong,1030.00,100060000,10294.0,High-tech R & D and manufacturing
3,2019,China,East-CN,3,East Coast,Zhejiang,904.00,56570000,15980.0,Multifunctional manufacturing: Light industria...
4,2019,China,Middle-CN,5,Yellow River Upper and Middle Reaches,Henan,787.00,95590000,8233.0,"Coal mining and coal deep processing, natural ..."
5,2019,China,West-CN,6,Yangtze River Upper and Middle Reaches,Sichuan,676.00,83020000,8143.0,"Rice and cotton, steel and non-ferrous metallu..."
6,2019,China,Middle-CN,6,Yangtze River Upper and Middle Reaches,Hubei,664.00,59020000,11250.0,"Rice and cotton, steel and non-ferrous metallu..."
7,2019,China,East-CN,4,Southeast Coast,Fujian,615.00,39110000,15725.0,"Export-oriented, digesting foreign advanced te..."
8,2019,China,Middle-CN,6,Yangtze River Upper and Middle Reaches,Hunan,576.00,68600000,8397.0,"Rice and cotton, steel and non-ferrous metallu..."
9,2019,China,East-CN,3,East Coast,Shanghai,553.00,24180000,22870.0,Multifunctional manufacturing: Light industria...


# Get data of USA

In [6]:
df_gdp_usa.head()

,State,Region,Nominal GDP (Billion)
0,Connecticut,New England,288.99
1,Maine,New England,68.44
2,Massachusetts,New England,604.21
3,New Hampshire,New England,89.84
4,Rhode Island,New England,64.44


In [7]:
df_pop_usa.head()

,State,Population
0,California,39512223
1,Texas,28995881
2,Florida,21477737
3,New York,19453561
4,Pennsylvania,12801989


In [21]:
# Strip leading and trailng white space in State column
df_gdp_usa['State'] = df_gdp_usa['State'].str.strip()
df_pop_usa['State'] = df_pop_usa['State'].str.strip()

# Merge dataframe for China
df_usa = pd.DataFrame.merge(df_gdp_usa,df_pop_usa,how='left',on='State')

# Add column
df_usa['Country'] = 'United States'
df_usa['Year'] = '2019'

# Rename column
df_usa.rename(columns={'State':'Administrative Division'}, inplace=True)

# Re-order Columns
df_usa = df_usa[['Year','Country','Region','Administrative Division','Nominal GDP (Billion)','Population']]

# Calculate GDP per Capita
df_usa['GDP per Capita'] = (df_usa['Nominal GDP (Billion)'] / df_usa['Population'])*1000000000

# Set 0 decimal places
df_usa['GDP per Capita'] = df_usa['GDP per Capita'].round(0)

# Concat 'US' to all value in[Region]
df_usa['Region'] = df_usa['Region'].map(str) + '-US'

df_usa.dtypes

Year                        object
Country                     object
Region                      object
Administrative Division     object
Nominal GDP (Billion)      float64
Population                   int64
GDP per Capita             float64
dtype: object

# Concat

In [9]:
# Concat data frame
clean_df = pd.concat([df_china, df_usa])

# Reset index
clean_df.reset_index(drop = True, inplace = True)

clean_df.head()

,Administrative Division,Characteristic,Country,Economic Zone,Economic Zone #,GDP per Capita,Nominal GDP (Billion),Population,Region,Year
0,Guangdong,"Export-oriented, digesting foreign advanced te...",China,Southeast Coast,4.0,13976.0,1561.0,111690000,East-CN,2019
1,Jiangsu,Multifunctional manufacturing: Light industria...,China,East Coast,3.0,17985.0,1444.0,80290000,East-CN,2019
2,Shandong,High-tech R & D and manufacturing,China,North Coast,2.0,10294.0,1030.0,100060000,East-CN,2019
3,Zhejiang,Multifunctional manufacturing: Light industria...,China,East Coast,3.0,15980.0,904.0,56570000,East-CN,2019
4,Henan,"Coal mining and coal deep processing, natural ...",China,Yellow River Upper and Middle Reaches,5.0,8233.0,787.0,95590000,Middle-CN,2019


In [10]:
clean_df.to_csv('clean_data/clean_china_and_usa.csv')

# Get data of Europe

In [11]:
df_continent.head()

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0


In [12]:
# Clean data in ['Country']
df_continent['Country'] = df_continent['Country_Name'].str.rsplit(',').str[0] 
df_continent.head()

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number,Country
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0,Afghanistan
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0,Albania
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0,Antarctica (the territory South of 60 deg S)
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0,Algeria
4,Oceania,OC,American Samoa,AS,ASM,16.0,American Samoa


In [13]:
df_WEO.head()

,Country,Subject Descriptor,Units,Scale,2017,2018,2019,2020,2021,2022,2023,2024,Estimates Start After
0,Afghanistan,"Gross domestic product, current prices",U.S. dollars,Billions,20.24,19.63,18.73,18.86,20,21.54,23.24,25.19,2018.0
1,Afghanistan,"Gross domestic product per capita, current prices",U.S. dollars,Units,569.53,544.98,513.11,509.76,533.09,566.42,602.88,644.95,2016.0
2,Afghanistan,"Inflation, average consumer prices",Index,NaN,111,111.69,114.58,119.74,125.43,131.7,138.28,145.2,2018.0
3,Afghanistan,Unemployment rate,Percent of total labor force,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,Population,Persons,Millions,35.53,36.02,36.51,37,37.51,38.03,38.54,39.06,2016.0


In [14]:
# Select columns based on condition
df_gdp_world = df_WEO[['Country','Units','Scale','2019']][df_WEO['Subject Descriptor'] == 'Gross domestic product, current prices']#[df_WEO['2019']]
df_gdp_world.head()

,Country,Units,Scale,2019
0,Afghanistan,U.S. dollars,Billions,18.73
5,Albania,U.S. dollars,Billions,15.42
10,Algeria,U.S. dollars,Billions,172.78
15,Angola,U.S. dollars,Billions,91.53
20,Antigua and Barbuda,U.S. dollars,Billions,1.69


In [29]:
# Merge dataframe 
df_gdp_world_ontinent = pd.DataFrame.merge(df_gdp_world,df_continent,how='left',on='Country')

# Select Europe gdp data
df_gdp_europe = df_gdp_world_ontinent[['Country','2019']][df_gdp_world_ontinent['Continent_Name'] == 'Europe']

# Rename column
df_gdp_europe.rename(columns={'2019':'Nominal GDP (Billion)'}, inplace=True)

# Merge dataframe 
df_europe = pd.DataFrame.merge(df_gdp_europe,df_pop_europe,how='left',on='Country')

# Add column
df_europe['Year'] = '2019'

# Change data type
df_europe['Nominal GDP (Billion)'] = pd.to_numeric(df_europe['Nominal GDP (Billion)'], errors='coerce') # .astype() not working
df_europe['Population'].astype(float)

# Calculate GDP per Capita
df_europe['GDP per Capita'] = (df_europe['Nominal GDP (Billion)'] / df_europe['Population'])*1000000000

# Set 0 decimal places
df_europe['GDP per Capita'] = df_europe['GDP per Capita'].round(0)

df_europe.head()

,Country,Nominal GDP (Billion),Population,Year,GDP per Capita
0,Albania,15.42,2862427.0,2019,5387.0
1,Armenia,13.44,2965269.0,2019,4532.0
2,Austria,447.72,8858775.0,2019,50540.0
3,Azerbaijan,47.17,9981457.0,2019,4726.0
4,Belarus,62.57,9475174.0,2019,6604.0


In [30]:
df_europe.to_csv('clean_data/clean_europe.csv')